## Breitzman 9/6/2022
<br>
Port of my 2013 Python 2.7 code to Python 3.x
<br>

Also borrows some code from Badal Juneja's port of my earlier code.

In [1]:
#what are we running?
import sys
print(sys.version)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


In [2]:
#need conda install -c conda-forge google-api-python-client
#or equivalent if using pip
import apiclient # google's python api library

In [3]:
from IPython.core.display import HTML  #this is useful for printing nice HTML within Jupyter notebook

In [4]:
import config  #fill config.py with APIkeys and put in same directory
GdevKey = config.GdevKey 

In [5]:
#here are some custom search engine keys
#you can use mine or make your own
patdevcx = '016938528964600140909:i6wj7bxhnra'  #custom search engine for patents
Gdevcx = '016938528964600140909:wupywukfgda'  #general google search engine
GSdevcx = '016938528964600140909:agrqxfwuuuq'  #google scholar
GLIdevcx = '016938528964600140909:m1qf0rkoow4'  #google linked-in
GPRdevcx = '016938528964600140909:zkjzldc4wwa'   #press releases
GVCdevcx = '016938528964600140909:zkjzldc4wwa'   #venture capital

In [6]:
## Here's some slick code from a former student (Badal Juneja) that does a nice job of printing the results
def print_html_formatted_result(result, print_count_start):
    
    '''
    Helper function for pretty printing google search api results
    '''
    
    for counter, item in enumerate(result['items'], print_count_start):
        display(HTML('<a href="{}">{}. {}</a><br>{}<br><br>'.format(
            item['link'], counter, item['htmlTitle'], item['htmlSnippet']
            )))

In [7]:
## Also modified from Badal.  Modification includes setting default search engine to regular google
def search_and_print(query, api_key, search_engine_key=Gdevcx, number_of_desired_results=10):
    
    '''
    Takes up to four arguments:
    query: as you will type them in google.com
    api_key: your developer key
    search_engine_key (optional): your personal search engine. Default is google.com with search all web turned on 
    number_of_desired_results (optional): number of results you want. Deafult is 10. Keep less than 50 to not use too many queries. 
    '''

    service = apiclient.discovery.build("customsearch", "v1", developerKey=api_key)
    resource = service.cse()

    number_of_results_printed = 0

    while number_of_results_printed < number_of_desired_results:

        left_to_print = number_of_desired_results - number_of_results_printed
        left_to_print = 10 if left_to_print >= 10 else left_to_print
        print_count_start = number_of_results_printed + 1

        result = resource.list(q=query, cx=search_engine_key, num=left_to_print, start=print_count_start).execute()
        
        if 'items' in result: # only if results found
            print_html_formatted_result(result, print_count_start)
        else:
            break
        
        number_of_results_printed += left_to_print

In [10]:
search_and_print('samsung', GdevKey, number_of_desired_results=11)

In [11]:
## Now watch how results change when we change the search engine to papers.
search_and_print('samsung', GdevKey, GSdevcx, number_of_desired_results=11)


In [12]:
## Now watch how results change when we change the search engine to Linked-in.
search_and_print('samsung', GdevKey, GLIdevcx, number_of_desired_results=11)

In [13]:
## Here's the non-Badal version.  This is useful if you want to put the results in a file or database.
## Note we can easily visit the URL and get the full text, but we'll just print the snippet
def search_and_print_oldFashioned(query, api_key, search_engine_key=Gdevcx, number_of_desired_results=10):
    
    '''
    Takes up to four arguments:
    query: as you will type them in google.com
    api_key: your developer key
    search_engine_key (optional): your personal search engine. Default is google.com with search all web turned on 
    number_of_desired_results (optional): number of results you want. Deafult is 10. Keep less than 50 to not use too many queries. 
    '''

    service = apiclient.discovery.build("customsearch", "v1", developerKey=api_key)
    resource = service.cse()

    number_of_results_printed = 0
    header_printed = False
    while number_of_results_printed < number_of_desired_results:

        left_to_print = number_of_desired_results - number_of_results_printed
        left_to_print = 10 if left_to_print >= 10 else left_to_print
        print_count_start = number_of_results_printed + 1

        result = resource.list(q=query, cx=search_engine_key, num=left_to_print, start=print_count_start).execute()
        
        if (not header_printed):  #this is a kludge so that we only print this part once
          print('SearchTerms: ' + result['queries']['request'][0]['searchTerms']+'\n')
          print('# of results: '+ result['queries']['request'][0]['totalResults'] + '\n')
          header_printed = True
        
        if 'items' in result: # only if results found
           for counter, item in enumerate(result['items'], print_count_start):
            print(str(counter)+'\nURL:'+str(item['link'])+'\nPage Title:'+item['htmlTitle']+'\nSnippet:'+item['htmlSnippet'])
        else:
            break
        
        number_of_results_printed += left_to_print
        
        

In [14]:
search_and_print_oldFashioned('photovoltaic "printed electronics"', GdevKey, number_of_desired_results=5)

SearchTerms: photovoltaic "printed electronics"

# of results: 164000

1
URL:https://www.printedelectronicsworld.com/tag/7/photovoltaics
Page Title:<b>Photovoltaics</b> | <b>Printed Electronics</b> World
Snippet:Using Transparent <b>Solar Power</b> for Cultivating Crops &middot; IDTechEx Research Subscribers ... <b>Printed Electronics</b> World. Aug 17, 2022&nbsp;...
2
URL:https://www.dupont.com/mobility-materials/session-overviews-printed-electronics-seminar.html
Page Title:DuPont <b>Photovoltaic</b> &amp; Advanced Materials Hosts &quot;Introduction to ...
Snippet:The development of <b>printed electronics</b> technology is described with reference to the early applications which it was initially used for including Membrane&nbsp;...
3
URL:https://www.sigmaaldrich.com/US/en/technical-documents/technical-article/materials-science-and-engineering/organic-electronics/polymer-based-materials
Page Title:Polymer-based Materials for <b>Printed Electronics</b>: Enabling High ...
Snippet:Polymer

## These could use some cleanup with beautiful soup but you get the idea
<br>

## we'll do some simpler cleanup

In [15]:
import html
s = 'Learn <b>data science</b> online today. Advance your career as a <b>data scientist</b> with <br> free courses from the world&#39;s top institutions. Join now.'
print(s,'\n\n',html.unescape(s),'\n')

Learn <b>data science</b> online today. Advance your career as a <b>data scientist</b> with <br> free courses from the world&#39;s top institutions. Join now. 

 Learn <b>data science</b> online today. Advance your career as a <b>data scientist</b> with <br> free courses from the world's top institutions. Join now. 



In [16]:
def clean5(s):
    s = html.unescape(s)  #this replaces unicode characters with the closest ascii fit.
    s = s.replace('<b>','')
    s = s.replace('</b>','')
    s = s.replace('<br>','')
    s = s.replace('  ',' ') #replace double spaces with single spaces
    return(s)

In [17]:
s = 'Learn <b>data science</b> online today. Advance your career as a <b>data scientist</b> with <br> free courses from the world&#39;s top institutions. Join now.'
print(s,'\n\n',clean5(s),'\n')

Learn <b>data science</b> online today. Advance your career as a <b>data scientist</b> with <br> free courses from the world&#39;s top institutions. Join now. 

 Learn data science online today. Advance your career as a data scientist with free courses from the world's top institutions. Join now. 

